# Bringing your own GPU
This predictor demonstrates how to bring your own GPU for making predictions on Function. Function integrates with [Runhouse](https://www.run.house/) to send your predictor code to your own custom GPU instances.

> For startups, you'll find this especially useful if you have credits with the major cloud providers (e.g. AWS, Azure, GCP).

> For enteprirse workloads, we instead recommend bringing your own cluster or running Function on-prem. [Reach out](mailto:sales@fxn.ai) to learn more.

## Installing Runhouse
First, we'll need to install `rsync`:

In [ ]:
!apt-get install -y rsync

Now let's install Runhouse:

In [ ]:
# Install RH
%pip install runhouse

To login to RH, we'll need a Runhouse token. Head over to [your Runhouse account](https://www.run.house/account) to generate your RH token.

Next, create a Function environment variable with your Runhouse token. Open a terminal and run the following command:
```bash
# Create an environment variable for your Runhouse token
fxn env create RH_TOKEN <Your Runhouse token>
```

Now, our predictor can login:

In [5]:
import runhouse as rh
from os import environ

rh.login(
    token=environ["RH_TOKEN"],
    download_secrets=True,
    download_config=True,
    upload_config=True,
    upload_secrets=True,
    interactive=False
)

____              __                             @ @ @
           / __ \__  ______  / /_  ____  __  __________     []___
          / /_/ / / / / __ \/ __ \/ __ \/ / / / ___/ _ \   /    /\____    @@
         / _, _/ /_/ / / / / / / / /_/ / /_/ (__  )  __/  /_/\_//____/\  @@@@
        /_/ |_|\__,_/_/ /_/_/ /_/\____/\__,_/____/\___/   | || |||__|||   ||

Retrieve your token 🔑 here to use 🏃 🏠 Runhouse for secrets and artifact management: ]8;id=825588;https://api.run.house/dashboard/?option=token\https://api.run.house]8;;\

INFO | 2023-07-19 08:10:29,931 | Getting secrets from Vault.
INFO | 2023-07-19 08:10:35,332 | Found credentials in shared credentials file: ~/.aws/credentials


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

I 07-19 08:10:37 aws_catalog.py:119] Fetching availability zones mapping for AWS...
INFO | 2023-07-19 08:10:39,158 | Found credentials in shared credentials file: ~/.aws/credentials
INFO | 2023-07-19 08:10:39,925 | Uploaded secrets for to Vault for: ['aws']
INFO | 2023-07-19 08:10:39,927 | Successfully logged into Runhouse.


In [6]:
!sky check

Checking credentials to enable clouds for SkyPilot.
  AWS: enabled          
  Azure: disabled          
    Reason: ~/.azure/msal_token_cache.json does not exist. Run the following commands:
      $ az login
      $ az account set -s <subscription_id>
    For more info: https://docs.microsoft.com/en-us/cli/azure/get-started-with-azure-cli
  GCP: disabled          
    Reason: GCP tools are not installed or credentials are not set. Run the following commands:
      $ pip install google-api-python-client
      $ conda install -c conda-forge google-cloud-sdk -y
      $ gcloud init
      $ gcloud auth application-default login
    For more info: https://skypilot.readthedocs.io/en/latest/getting-started/installation.html
  Lambda: disabled          
    Reason: Failed to access Lambda Cloud with credentials. To configure credentials, go to:
      https://cloud.lambdalabs.com/api-keys
    to generate API key and add the line
      api_key = [YOUR API KEY]
    to ~/.lambda_cloud/lambda_keys


## Creating an On-Demand Cluster
Now, we can create a cluster on our AWS account:

In [8]:
# Create an instance on AWS
rh.cluster(
    name="rh-a10x",
    instance_type="g5.2xlarge",
    provider="aws",
    autostop_mins=5,
    region="us-east-1"
).save()

INFO | 2023-07-19 08:14:31,753 | Attempting to load config for /olokobayusuf/rh-a10x from RNS.
INFO | 2023-07-19 08:14:31,988 | No config found in RNS: {'detail': 'Resource does not exist'}
INFO | 2023-07-19 08:14:31,990 | Saving config to RNS: {'name': '/olokobayusuf/rh-a10x', 'resource_type': 'cluster', 'resource_subtype': 'OnDemandCluster', 'instance_type': 'g5.2xlarge', 'num_instances': None, 'provider': 'aws', 'autostop_mins': 5, 'use_spot': False, 'image_id': None, 'region': 'us-east-1', 'sky_state': None}
INFO | 2023-07-19 08:14:32,225 | Saving new resource in RNS for Runhouse URI <resource/olokobayusuf:rh-a10x>


## Implementing the Predictor
Let's create a predictor that uses StableDiffusion to generate an image from a text `prompt`. Unlike the standard `stable-diffusion` sample, here will be running the inference on a GPU in our own AWS account:

In [9]:
def stable_diffusion_generate (prompt: str):
    # According to RH, imports must be inside the function
    from diffusers import StableDiffusionPipeline
    from torch import float16
    # Create the SD pipeline
    pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-base", torch_dtype=float16, revision="fp16")
    # Send to the GPU
    pipeline.to("cuda")
    # Generate an image
    result = pipeline(prompt).images[0]
    # Return
    return result

INFO | 2023-07-19 08:22:45,046 | Attempting to load config for /olokobayusuf/rh-a10x from RNS.


The `stable_diffusion_generate` function will be sent to our GPU cluster for inference. With this, we can now write our Function predictor:

In [ ]:
# Get our GPU cluster
gpu = rh.cluster(name="rh-a10x")

# Send the `stable_diffusion_generate` function to our GPU cluster using RH
stable_diffusion_generate = rh.function(fn=stable_diffusion_generate).to(
    gpu,
    env=["./", "torch --upgrade --extra-index-url https://download.pytorch.org/whl/cu117", "diffusers", "transformers"]
)

# Define the Function predictor
def predict (prompt: str):
    # Generate an image by sending to our custom cluster
    result = stable_diffusion_generate(prompt)
    # Return the result
    return result

## Creating the Predictor on Function
Because we're bringing our own GPU, we can create the predictor on Function and use CPU acceleration:
```bash
fxn create @username/byo-gpu bring-your-gpu.ipynb --overwrite
```

> Replace `username` with your Function username.